<a href="https://colab.research.google.com/github/Ian-Mutuge/ConserveAI/blob/master/Object_detection_YOLOV11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
%pip install torch torchvision torchaudio
!pip install opencv-python-headless
import matplotlib.pyplot as plt

Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.0/235.7 GB disk)


In [2]:
from ultralytics import YOLO
import os
import cv2
import shutil
import pandas as pd
from pathlib import Path

In [3]:
model = YOLO("yolo11s.pt")

100%|██████████| 18.4M/18.4M [00:00<00:00, 83.5MB/s]


In [4]:
!git clone https://github.com/NanoCode012/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format.git

Cloning into 'OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format'...
remote: Enumerating objects: 548, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 548 (delta 2), reused 5 (delta 2), pack-reused 542 (from 1)
Receiving objects: 100% (548/548), 34.16 MiB | 20.94 MiB/s, done.
Resolving deltas: 100% (218/218), done.


In [5]:
!pip3 install -r /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.18.1
    Uninstalling docutils-0.18.1:
      Successfully uninstalled docutils-0.18.1


In [6]:
!mkdir OID

In [7]:
!mkdir OID/Dataset


In [8]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [9]:
# Define classes and download parameters
classes = "Axe Dagger Handgun Knife Rifle Shotgun Sword Elephant Giraffe Lion Rhinoceros Zebra"
samples = 70

!python /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/main.py downloader --classes {classes} --type_csv test --limit {samples} --yoloLabelStyle

    [INFO] |  saving dataset configurations at ./OID/Dataset/config.json

		   ___   _____  ______            _    _       _    _  
		 .'   `.|_   _||_   _ `.         | |  | |     | |  | |    
		/  .-.  \ | |    | | `. \ _   __ | |__| |_    |  | | |   
		| |   | | | |    | |  | |[ \ [  ]|____   _|     | | | 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_       | |
		 `.___.'|_____||______.'   \__/     |_____|     |_|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Axe.
   [ERROR] | Missing the class-descriptions-boxable.cs

In [10]:
# Define classes and download parameters
classes = "Axe Dagger Handgun Knife Rifle Shotgun Sword Elephant Giraffe Lion Rhinoceros Zebra"
samples = 100

!python /content/OIDv6_ToolKit_Download_Open_Images_Support_Yolo_Format/main.py downloader --classes {classes} --type_csv train --limit {samples} --yoloLabelStyle

    [INFO] |  saving dataset configurations at ./OID/Dataset/config.json

		   ___   _____  ______            _    _       _    _  
		 .'   `.|_   _||_   _ `.         | |  | |     | |  | |    
		/  .-.  \ | |    | | `. \ _   __ | |__| |_    |  | | |   
		| |   | | | |    | |  | |[ \ [  ]|____   _|     | | | 
		\  `-'  /_| |_  _| |_.' / \ \/ /     _| |_       | |
		 `.___.'|_____||______.'   \__/     |_____|     |_|
	

             _____                    _                 _             
            (____ \                  | |               | |            
             _   \ \ ___  _ _ _ ____ | | ___   ____  _ | | ____  ____ 
            | |   | / _ \| | | |  _ \| |/ _ \ / _  |/ || |/ _  )/ ___)
            | |__/ / |_| | | | | | | | | |_| ( ( | ( (_| ( (/ /| |    
            |_____/ \___/ \____|_| |_|_|\___/ \_||_|\____|\____)_|    
                                                          
        
    [INFO] | Downloading Axe.
   [ERROR] | Missing the train-annotations-bbox.csv fi

In [11]:
import torch
torch.cuda.empty_cache()


In [12]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [13]:
model.train(
    data="/content/weapon_dataset.yaml",
    epochs=100,
    imgsz=640,
    augment=True,
    hsv_h=0.015,       # Small hue variation
    hsv_s=0.7,         # Saturation variation
    hsv_v=0.4,         # Brightness variation
    degrees=10.0,      # Minor rotation
    translate=0.1,     # Slight translation
    scale=0.5,         # Moderate scaling
    mosaic=1.0         # Mosaic for background variety
)



Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/weapon_dataset.yaml, epochs=100, time=None, patience=20, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_box

100%|██████████| 755k/755k [00:00<00:00, 21.1MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytic

100%|██████████| 5.35M/5.35M [00:00<00:00, 90.1MB/s]


AMP: checks passed ✅


...

     91/100      4.34G     0.4743     0.4682     0.9747         41        640: 100%|██████████| 75/75 [00:34<00:00,  2.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:07<00:00,  2.83it/s]

                   all        701        989      0.592      0.529      0.535      0.404
EarlyStopping: Training stopped early as no improvement observed in last 20 epochs. Best results observed at epoch 71, best model saved as best.pt.
To update EarlyStopping(patience=20) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



91 epochs completed in 1.080 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 19.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 19.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11s summary (fused): 238 layers, 9,417,444 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 22/22 [00:11<00:00,  1.89it/s]


                   all        701        989      0.517      0.612      0.538      0.409
                   Axe          2          2     0.0426        0.5      0.168      0.167
                Dagger         70        101      0.311      0.564      0.376      0.286
               Handgun         70         78      0.534      0.654      0.603      0.499
                 Knife         70         95      0.515      0.559      0.466      0.346
                 Rifle         70         96      0.607      0.302      0.444       0.28
               Shotgun         58         66      0.282      0.621      0.314      0.187
                 Sword         66         89      0.483      0.438      0.422       0.33
              Elephant         70        120      0.469      0.619       0.58      0.384
               Giraffe         47         73       0.75      0.712      0.729      0.543
                  Lion         70         76      0.843      0.908      0.936      0.744
            Rhinocero

In [14]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each categor

Ultralytics 8.3.25 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11s summary (fused): 238 layers, 9,417,444 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/OID/Dataset/test/Axe/labels.cache... 701 images, 0 backgrounds, 0 corrupt: 100%|██████████| 701/701 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 44/44 [00:11<00:00,  3.67it/s]


                   all        701        989      0.511      0.624       0.57      0.438
                   Axe          2          2     0.0457        0.5      0.498      0.497
                Dagger         70        101      0.303      0.595      0.378      0.287
               Handgun         70         78      0.519      0.654      0.627      0.521
                 Knife         70         95      0.505      0.579      0.473      0.349
                 Rifle         70         96      0.589      0.314       0.45       0.28
               Shotgun         58         66      0.269      0.624      0.306      0.178
                 Sword         66         89      0.467      0.473      0.427      0.327
              Elephant         70        120      0.465      0.633      0.582      0.383
               Giraffe         47         73      0.737      0.712      0.729      0.545
                  Lion         70         76      0.851      0.895      0.938      0.743
            Rhinocero

array([    0.49733,     0.28742,     0.52112,     0.34913,     0.28003,     0.17845,     0.32706,     0.38331,     0.54479,     0.74325,     0.79706,     0.34827])

In [15]:
from IPython.display import HTML
import cv2
import os

In [ ]:
# Run model and save the processed video
results = model(source="/content/Sword.avi", show=True, conf=0.4, save=True)

# Check if the video file has been saved, then rename it to a specific name
output_video_path = "/content/runs/detect/exp/Sword.avi"  # Adjust if the save location is different
if os.path.exists(output_video_path):
    processed_video_path = "/content/processed_video.mp4"
    os.rename(output_video_path, processed_video_path)

# Function to display video in Colab
def display_video(video_path, width=640):
    return HTML(f"""
        <video width="{width}" controls>
            <source src="{video_path}" type="video/mp4">
        </video>
    """)

# Display the processed video
display_video("/content/processed_video.mp4")



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

